<a href="https://colab.research.google.com/github/JuliaOrtheden/COVID19-research-challenge/blob/master/dhs_ridge_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-cmicgthn
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-cmicgthn
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 50 kB 3.0 MB/s 
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-py3-none-any.whl size=20045 sha256=888336eee38f5667cf97f118b68b4b6435523924e152abda8d415f3825d880f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-bhgaz10m/wheels/0b/96/56/27b17c903efc647c51e4f364bfc20aa67f8d3dccad63c4fb4e
Successfully built image-classifiers


In [ ]:
import numpy as np
import pandas as pd
import pickle
import os

import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import sklearn.linear_model

from collections import defaultdict
from glob import glob
import re
import sys
import tensorflow as tf
from classification_models.tfkeras import Classifiers

In [ ]:
dhs_clusters = pd.read_csv('dhs_clusters.csv')

In [ ]:
dhs_ooc_folds = pd.read_pickle('dhs_ooc_folds.pkl')

In [ ]:
angola_indices = dhs_clusters[dhs_clusters['country']=='angola'].index
all_countries_indices = dhs_clusters[dhs_clusters['country']!='angola'].index

In [ ]:
all_countries = dhs_clusters['country'].unique()

In [ ]:
def get_country_indices(country):
  return dhs_clusters[dhs_clusters['country']==country].index

def get_all_country_indices():
  return  dhs_clusters.index

In [ ]:
get_country_indices('angola')

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            959, 960, 961, 962, 963, 964, 965, 966, 967, 968],
           dtype='int64', length=969)

In [ ]:
def train_linear_logo(features, labels, group_labels, cv_groups, test_groups,
                      weights=None, linear_model=sklearn.linear_model.Ridge,
                      plot=True, group_names=None, return_weights=False,
                      verbose=False):
    '''Leave-one-group-out cross-validated training of a linear model.
    Args
    - features: np.array, shape [N, D]
        each feature dim should be normalized to 0 mean, unit variance
    - labels: np.array, shape [N]
    - group_labels: np.array, shape [N], type np.int32
    - cv_groups: list of int, labels of groups to use for LOGO-CV
    - test_groups: list of int, labels of groups to test on
    - weights: np.array, shape [N]
    - linear_model: sklearn.linear_model
    - plot: bool, whether to plot MSE as a function of alpha
    - group_names: list of str, names of the groups, only used when plotting
    - return_weights: bool, whether to return the final trained model weights
    - verbose: bool
    Returns
    - test_preds: np.array, predictions on indices from test_groups
    - coefs: np.array, shape [D] (only returned if return_weights=True)
    - intercept: float (only returned if return_weights=True)
    '''
    cv_indices = np.isin(group_labels, cv_groups).nonzero()[0]
    test_indices = np.isin(group_labels, test_groups).nonzero()[0]

    X = cv_indices #features[cv_indices]
    y = labels[cv_indices]
    groups = group_labels[cv_indices]
    w = None if weights is None else weights[cv_indices]

    alphas = 2**np.arange(-5, 35, 3.0)
    preds = np.zeros([len(alphas), len(cv_indices)], dtype=np.float64)
    group_mses = np.zeros([len(alphas), len(cv_groups)], dtype=np.float64)
    leftout_group_labels = np.zeros(len(cv_groups), dtype=np.int32)
    logo = sklearn.model_selection.LeaveOneGroupOut()

    for i, alpha in enumerate(alphas):
        if verbose:
            print(f'\rAlpha: {alpha} ({i+1}/{len(alphas)})', end='')

        # set random_state for deterministic data shuffling
        model = linear_model(alpha=alpha, random_state=123)

        for g, (train_indices, val_indices) in enumerate(logo.split(X, groups=groups)):
            train_X, val_X = X[train_indices], X[val_indices]
            train_y, val_y = y[train_indices], y[val_indices]
            train_w = None if w is None else w[train_indices]
            val_w = None if w is None else w[val_indices]
            model.fit(X=train_X, y=train_y, sample_weight=train_w)
            val_preds = model.predict(val_X)
            preds[i, val_indices] = val_preds
            group_mses[i, g] = np.average((val_preds - val_y) ** 2, weights=val_w)
            leftout_group_labels[g] = groups[val_indices[0]]

    if verbose:
        print()
    mses = np.average((preds - y) ** 2, axis=1, weights=w)  # shape [num_alphas]

    if plot:
        h = max(3, len(group_names) * 0.2)
        fig, ax = plt.subplots(1, 1, figsize=[h*2, h], constrained_layout=True)
        for g in range(len(cv_groups)):
            group_name = group_names[leftout_group_labels[g]]
            ax.scatter(x=alphas, y=group_mses[:, g], label=group_name,
                       c=[cm.tab20.colors[g % 20]])
        ax.plot(alphas, mses, 'g-', label='Overall val mse')
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Left-out Group')
        ax.set(xlabel='alpha', ylabel='mse')
        ax.set_xscale('log')
        ax.grid(True)
        plt.show()

    best_alpha = alphas[np.argmin(mses)]
    best_model = linear_model(alpha=best_alpha)
    best_model.fit(X=X, y=y, sample_weight=w)
    test_X, test_y, = features[test_indices], labels[test_indices]
    test_preds = best_model.predict(test_X)

    best_val_mse = np.min(mses)
    test_w = None if weights is None else weights[test_indices]
    test_mse = np.average((test_preds - test_y) ** 2, weights=test_w)
    print(f'best val mse: {best_val_mse:.3f}, best alpha: {best_alpha}, test mse: {test_mse:.3f}')

    if not return_weights:
        return test_preds
    else:
        coefs = best_model.coef_
        intercept = best_model.intercept_
        return test_preds, coefs, intercept

In [ ]:
def run_ridgecv_keep():
    '''
    For every country C (the test country):
      1. uses leave-one-country-out CV on all other countries
         to tune ridge model alpha parameter
      2. using best alpha, trains ridge model on all countries except C
      3. runs trained ridge model on C
    Saves predictions for each country on test.

    Args
    - model_name: str, format 'resnet_{bands}', e.g. 'resnet_ms'
    - labels: np.array, shape [num_examples]
    - locs: np.array, shape [num_examples, 2]
    - country_labels: np.array, shape [num_examples]
    - folds: dict, fold (str) => dict
    - keep_frac: float, fraction of non-test-country data to use for training
    - seed: int
    - savedir: str
    '''
    num_examples = len(all_countries)
    test_preds = np.zeros(num_examples, dtype=np.float32)
    FOLDS = ['A', 'B', 'C', 'D', 'E']
    all_countries_set = get_all_country_indices()

    countries_to_indices = defaultdict(list)
    for i, country in enumerate(all_countries):
        countries_to_indices[country].append(i)


    for f in FOLDS:
        for test_country in dhs_ooc_folds[f]['test']:
            print('test country:', test_country)

            test_country_set = {test_country}
            cv_countries_set = all_countries_set - test_country_set
            test_indices = get_indices_for_countries(test_country_set)
            test_preds[test_indices] = train_linear_logo(
                features=features[keep_subset_indices],
                labels=labels[keep_subset_indices],
                group_labels=country_labels[keep_subset_indices],
                cv_groups=countries_to_nums(cv_countries_set),
                test_groups=countries_to_nums(test_country_set),
                plot=False,
                group_names=COUNTRIES)

    # save preds on the test set
    os.makedirs(savedir, exist_ok=True)
    npz_path = os.path.join(savedir, 'test_preds_keep{k}_seed{s}.npz'.format(k=keep_frac, s=seed))
    print('Saving preds to:', savedir)
    np.savez_compressed(npz_path, test_preds=test_preds, labels=labels, locs=locs)
    '''
    for test_country in all_countries:
        print('test country:', test_country)

        test_country_set = {}
        cv_countries_set = all_countries_indices
        test_indices = angola_indices #get_indices_for_countries(test_country_set)
        ResNet18, _  = Classifiers.get('resnet18')
        img_net = ResNet18((224, 224, 3), weights='imagenet')
        features = img_net.layers[0].get_weights()
        test = train_linear_logo(
                    features= features,
                    labels=dhs_ooc_folds['A'],
                    group_labels=all_countries,
                    cv_groups=all_countries_indices,
                    test_groups=angola_indices,
                    plot=False,
                    group_names=all_countries)
        print(test)
    '''

In [ ]:
run_ridgecv_keep()

TypeError: ignored

In [ ]:
def load_npz(path, verbose=True, check=None):
    '''Loads .npz file into a dict.
    Args
    - path: str, path to .npz file
    - verbose: bool, whether to print out type and shape info
    - check: dict, key (str) => np.array, values to check
    Returns
    - result: dict
    '''
    result = {}
    with np.load(path) as npz:
        for key, value in npz.items():
            result[key] = value
            if verbose:
                print('{k}: dtype={d}, shape={s}'.format(k=key, d=value.dtype, s=value.shape))
    if check is not None:
        for key in check:
            assert key in result
            assert np.allclose(check[key], result[key])
    return result

In [ ]:
file_path = 'dhs_image_hists.npz'
npz = load_npz(file_path)

labels = npz['labels']
locs = npz['locs']
years = npz['years']

num_examples = len(labels)

image_hists: dtype=int64, shape=(19669, 8, 102)
labels: dtype=float32, shape=(19669,)
locs: dtype=float32, shape=(19669, 2)
years: dtype=int32, shape=(19669,)
nls_center: dtype=float32, shape=(19669,)
nls_mean: dtype=float32, shape=(19669,)


In [ ]:
npz

{'image_hists': array([[[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0]],
 
        [[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0]],
 
        [[ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0]],
 
        ...,
 
        [[ 0,  0,  0, ...,  2,  1, 27],
         [ 0,  0,  0, ...,  0,  5, 27],
         [ 0,  0,  0, ...,  0,  0, 10],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...